### Model 1 Naiye Baeyes

In [1]:
import pandas as pd

In [2]:
# df=pd.read_csv('pre_title_steaming.csv')
df=pd.read_csv('pre_title.csv')

In [3]:
df.head(10)

,title,url_berita,sentimen
0,strategi sleman dorong ekonomi pakai batik int...,https://20.detik.com/blak-blakan/20231204-2312...,1
1,greysia polii ajak anak muda terus melangkah r...,https://20.detik.com/demi-indonesia/20231027-2...,1
2,pan x jakcloth pesta anak nongkrong sambangi b...,https://20.detik.com/advertorial/20231109-2311...,1
3,arahan megawati rakernas iv pdip turun rakyatm...,https://20.detik.com/detikupdate/20231001-2310...,1
4,erick thohir ajak muda tak mager indonesia leb...,https://20.detik.com/demi-indonesia/20231027-2...,1
5,pdip sebut ganjar muliakan petani nelayan jadi...,https://20.detik.com/detikupdate/20231001-2310...,1
6,bertemu habib rizieq cak imin bantah bahas duk...,https://20.detik.com/detikupdate/20231001-2310...,1
7,kades bandung mundur dukung amin anies salut,https://20.detik.com/detikupdate/20231001-2310...,1
8,gestur prabowo ditanya soal kans berduet ganjar,https://20.detik.com/detikupdate/20231001-2310...,1
9,anies bertemu alumni itb bahas metode ilmiah m...,https://20.detik.com/detikupdate/20231001-2310...,1


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [5]:
# Ubah nilai sentimen
def map_sentiment(sentiment):
    if sentiment == 1:
        return 'positif'
    elif sentiment == 0:
        return 'netral'
    else:
        return 'negatif'

# df = pd.read_csv('pre_title_steaming.csv')
# df = pd.read_csv('pre_title.csv')
df['sentimen'] = df['sentimen'].map(map_sentiment)

In [6]:
# Membagi data menjadi fitur dan label
X = df['title']
y = df['sentimen']

In [7]:
X

0        strategi sleman dorong ekonomi pakai batik int...
1        greysia polii ajak anak muda terus melangkah r...
2        pan x jakcloth pesta anak nongkrong sambangi b...
3        arahan megawati rakernas iv pdip turun rakyatm...
4        erick thohir ajak muda tak mager indonesia leb...
                               ...                        
14995    bagaimana cara cek tps pemilu 2024 simak infor...
14996    dinkes dki siap fasilitasi pemeriksaan kesehat...
14997    dilantik ksad jenderal maruli ungkap pesan jok...
14998    china imbau warganya segera tinggalkan perbata...
14999    zita anjani silap lidah manusiawi gibran sebut...
Name: title, Length: 15000, dtype: object

In [8]:
y

0        positif
1        positif
2        positif
3        positif
4        positif
          ...   
14995     netral
14996     netral
14997     netral
14998     netral
14999     netral
Name: sentimen, Length: 15000, dtype: object

In [9]:
def prepare_datasets(corpus, labels, test_data_proportion=0.3):
    train_X, test_X, train_Y, test_Y = train_test_split(corpus, labels,
    test_size=0.33,random_state=42)
    return train_X, test_X, train_Y, test_Y

norm_train_corpus, norm_test_corpus, train_labels, test_labels = prepare_datasets(X,y,test_data_proportion=0.3)

In [10]:
norm_train_corpus[0:10]

655      bawaslu terima laporan logistik pemilu tak dik...
12044    anggap banyak tekanan tpn ganjarmahfud sebut p...
14844    warga solo gugat mk minta gubernur u40 bisa ny...
13985           gempa m 20 guncang wonosobo berpusat darat
7974     alex marwata ungkap laporan korupsi kementan d...
8612     10000 orang tewas sekjen pbb gaza jadi kuburan...
6883     bagibagi susu sosialisasi program prabowogibra...
2199     aniesmuhaimin nomor urut 1 cak imin tandatanda...
2960     bawaslu imbau anggota lapor diintimidasi awasi...
4640                  7 fakta gencatan senjata 4 hari gaza
Name: title, dtype: object

In [11]:
norm_test_corpus[0:10]

11499    deretan anak presiden terjun politik gibran ah...
6475     firli tak hadiri panggilan polda metro hari ta...
13167           angkaangka korupsi apd pandemi bikin ngeri
862      fakta penemuan kerangka manusia berserakan per...
5970          firli bahuri hadiri sidang praperadilan hari
6706     kritik ruu dkj anies demokrasi harusnya maju b...
3017     anies selesaikan pejabat punya konflik kepenti...
3781     dirbinmas pmj dilarang politisasi tempat ibada...
3898     bupati bogor jadikan sasak panjang desa wisata...
2250     ott kpk tetapkan kajari bondowoso puji triasmo...
Name: title, dtype: object

In [12]:
#Fungsi untuk mengekstraksi feature menggunakan TF-IDF Model
def tfidf_transformer(bow_matrix):
    transformer = TfidfTransformer(norm='l2',
                                    smooth_idf=True,
                                    use_idf=True)
    tfidf_matrix = transformer.fit_transform(bow_matrix)
    return transformer, tfidf_matrix
def tfidf_extractor(corpus, ngram_range=(1,1)):
    vectorizer = TfidfVectorizer(min_df=1,
                                norm='l2',
                                smooth_idf=True,
                                use_idf=True,
                                ngram_range=ngram_range)
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features

In [13]:
#mengunakan model TF-IDF untuk mengekstraksi feature
tfidf_vectorizer, tfidf_train_features = tfidf_extractor(norm_train_corpus)
tfidf_test_features = tfidf_vectorizer.transform(norm_test_corpus)

In [14]:
from sklearn.linear_model import SGDClassifier
train_features=tfidf_train_features
train_labels=train_labels
test_features=tfidf_test_features
test_labels=test_labels

In [15]:
df_train = pd.DataFrame()
df_train['norm_train'] = norm_train_corpus
df_train['train_labels'] = train_labels
df_test = pd.DataFrame()
df_test['norm_test'] = norm_test_corpus
df_test['test_labels'] = test_labels

In [16]:
df_train.head()

,norm_train,train_labels
655,bawaslu terima laporan logistik pemilu tak dik...,positif
12044,anggap banyak tekanan tpn ganjarmahfud sebut p...,netral
14844,warga solo gugat mk minta gubernur u40 bisa ny...,netral
13985,gempa m 20 guncang wonosobo berpusat darat,netral
7974,alex marwata ungkap laporan korupsi kementan d...,negatif


In [17]:
df_test.head()

,norm_test,test_labels
11499,deretan anak presiden terjun politik gibran ah...,netral
6475,firli tak hadiri panggilan polda metro hari ta...,negatif
13167,angkaangka korupsi apd pandemi bikin ngeri,netral
862,fakta penemuan kerangka manusia berserakan per...,positif
5970,firli bahuri hadiri sidang praperadilan hari,negatif


In [18]:
train_features[0:1]

<1x11986 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [19]:
train_labels[0:1]

655    positif
Name: sentimen, dtype: object

In [20]:
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

In [21]:
# clf = SGDClassifier(loss='hinge', max_iter=100)
# clf = LinearSVC(penalty='l2', loss='squared_hinge', max_iter=100)
clf = MultinomialNB()

In [22]:
clf.fit(train_features, train_labels)
predictions = clf.predict(test_features)
print("Label test: \n" + format(test_labels[:10]))
print("Prediction test: \n"+ format(predictions[:10]))

Label test: 
11499     netral
6475     negatif
13167     netral
862      positif
5970     negatif
6706     negatif
3017     positif
3781     positif
3898     positif
2250     positif
Name: sentimen, dtype: object
Prediction test: 
['positif' 'negatif' 'negatif' 'netral' 'negatif' 'negatif' 'positif'
 'positif' 'negatif' 'negatif']


In [23]:
print("Accuration: "+format(clf.score(test_features,test_labels)))

Accuration: 0.6537373737373737


In [24]:
import numpy as np
from sklearn import metrics

In [25]:
def get_metrics(true_labels, predicted_labels):
    print('Accuracy: ', np.round(metrics.accuracy_score(true_labels,
                                                        predicted_labels),2))
    print('Precision: ', np.round(metrics.precision_score(true_labels,
                                                        predicted_labels,
                                                        average='weighted'),2))
    print('Recall: ', np.round(metrics.recall_score(true_labels,
                                                        predicted_labels,
                                                        average='weighted'),2))
    print('F1 Score: ', np.round(metrics.f1_score(true_labels,
                                                        predicted_labels,
                                                        average='weighted'),2))
# mengevaluasi performa prediksi menggunakan model
get_metrics(true_labels=test_labels,predicted_labels=predictions)

Accuracy:  0.65
Precision:  0.65
Recall:  0.65
F1 Score:  0.65


In [26]:
test = ('Jeka Saragih On Fire Menatap Debut di UFC')
line = tfidf_vectorizer.transform([test])
line.shape
test_result = clf.predict(line)
print("Hasil Prediksi:", test_result) 


Hasil Prediksi: ['negatif']
